In [1]:
#!pip install sqlalchemy
import os
import pandas as pd
import openpyxl
import glob

In [3]:
# cope datafram
data_input= combined_df.copy()
pmt = combined_pmt.copy()


In [66]:
combined_pmt.head()

,Project type,Project Owner,Project-ID,Product group,ISO county code,Contract signing date,Closing date
0,3rd,MaSG,MSG100018,Structural steel bearings,TW,2019-12-20,NaT
1,3rd,MaSG,MSG100103,Monitoring,PH,2020-07-08,2023-01-31
2,3rd,MaSG,MSG100104,Monitoring,PH,2020-07-08,2023-01-31
3,3rd,MaSG,MSG100125,Structural steel bearings,AE,2020-11-03,2022-08-31
4,3rd,MaSG,MSG100101,Structural steel bearings,SG,2020-10-20,NaT


In [4]:
#duplicate check and removal

def duplicate_check(df, df_name):
    duplicate_rows = df.duplicated().sum()
    if duplicate_rows == 0:
        print(f"No duplicates in '{df_name}'")
    else:
        df = df.drop_duplicates(keep='first')  
        print(f"Duplicates removed in '{df_name}': {duplicate_rows}")
    return df

In [5]:
# run function duplicate_checkdf = df[df[first_col].notna() & (df[first_col].astype(str).str.strip() != '')]
data_input = duplicate_check(data_input, 'data_input')
pmt = duplicate_check(pmt, 'pmt')

No duplicates in 'data_input'
No duplicates in 'pmt'


In [6]:
#name replacement
def clean_columns(df):
    df.columns = df.columns.str.replace(' ', '_').str.lower()
    df.columns = df.columns.str.replace('_-_', '_', regex=False)
    df.columns = df.columns.str.replace('-', '_').str.lower()
    df.columns = df.columns.str.replace('.', '', regex=False)
    return df

data_input = clean_columns(data_input)
pmt = clean_columns(pmt)

In [7]:
#delete column
#data_input = data_input.drop(data_input.columns[0], axis=1)
data_input = data_input.drop(data_input.columns[[0,2]], axis=1) 
data_input = data_input.rename(columns={'p_code': 'project_id'})

#data_input.head()

In [8]:
# datatype conversion
#from datetime import datetime

data_input= data_input.infer_objects() 

data_input['p_clo'] = pd.to_datetime(data_input['p_clo'], errors='coerce')

pmt['closing_date'] = pd.to_datetime(pmt['closing_date'], errors='coerce')
pmt['contract_signing_date'] = pd.to_datetime(pmt['contract_signing_date'], errors='coerce')

#data_input.info()


In [9]:
def round_numeric_columns(df, decimals=2):
    numeric_cols = df.select_dtypes(include=['number'])  
    
    numeric_cols = numeric_cols.fillna(0)
    
    df[numeric_cols.columns] = numeric_cols.round(decimals)
    
    return df
    
data_input = round_numeric_columns(data_input)

In [159]:
data_input.head()

,project_id,p_clo,p_sta,budcv,buddmc_3rd,buddmc_ic,buddmc_int,buddmc,budtsc_3rd,budtsc_ic,...,acttse,actpmc,acttra,actcom,actlic,actoth,actoc,actpc,actis,actap
0,MSG100018,NaT,Open,5993350.67,0.00,3853397.60,0.0,3853397.60,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.00,3369267.44,4115057.89,898358.27
1,MSG100103,2023-01-31,Closed,74792.04,0.00,29319.76,0.0,29319.76,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
2,MSG100104,2023-01-31,Closed,164570.91,0.00,80553.55,0.0,80553.55,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
3,MSG100125,2022-08-31,Closed,4554039.53,0.00,3177003.75,0.0,3177003.75,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,414473.94,414473.94,4856663.00,4954026.91,-0.00
4,MSG100101,NaT,Open,1600000.00,185004.56,634390.47,0.0,819395.04,0.0,0.0,...,72752.16,0.0,0.0,0.0,0.0,8809.07,8809.07,719228.84,783331.57,0.00
